In [1]:
import glob
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import matplotlib.pyplot as plt     
%matplotlib inline

In [2]:
image_path = glob.glob('Dataset/*/*')     #寻找数据集的路径
image_path[:3]                      #前三张图片的路径

['Dataset\\Hazardous_waste\\haz_bet1.jpeg',
 'Dataset\\Hazardous_waste\\haz_bet10.jpeg',
 'Dataset\\Hazardous_waste\\haz_bet100.jpeg']

In [3]:
all_label_name = [img_p.split('\\')[1] for img_p in image_path]     #获取类别名，要用[1]
label_name = np.unique(all_label_name)         #np.unique()函数是将重复的，去除重复类别名
print(label_name)

['Hazardous_waste' 'Kitchen_waste' 'Other_trash' 'Recyclable_trash']


In [4]:
#英文标签转为数字标签
label_to_index = dict((name, i) for i,name in enumerate(label_name))   #enumerrate生成(索引,名称)
label_to_index

{'Hazardous_waste': 0,
 'Kitchen_waste': 1,
 'Other_trash': 2,
 'Recyclable_trash': 3}

In [5]:
index_to_label = dict((f, q) for q,f in label_to_index.items())  #把上面的数字标签位置换一下，items
index_to_label                                               #并转化为dict

{0: 'Hazardous_waste',
 1: 'Kitchen_waste',
 2: 'Other_trash',
 3: 'Recyclable_trash'}

In [6]:
all_index = [label_to_index.get(name) for name in all_label_name]   #获取字典的键对值
all_index[-5:]      #展示最后五张键对值来check

[3, 3, 3, 3, 3]

In [7]:
# 随机打乱图片路径和标签
np.random.seed(2021)  #设置随机数
random_index = np.random.permutation(len(image_path))  #随机排序，index
image_path = np.array(image_path)[random_index]  #np.array主要是将数据转化成数组
all_index = np.array(all_index)[random_index]

In [8]:
num = int(len(image_path)*0.8)    #拿80%图片来训练，20%图片来做测试
image_train = image_path[:num]     #train dataset
index_train = all_index[:num]     #训练标签图片路径
image_val = image_path[num:]     #test dataset
index_val = all_index[num:]      #测试标签图片路径

In [9]:
#expand_dims的第二个参数，主要是选择在第几维度增加维度，-1即在最后一维增加维度
#这里是index_train,而不是train_path
index_train = tf.expand_dims(index_train, -1)     #给训练图片索引增加一个维度
index_val = tf.expand_dims(index_val, -1)        #给测试集索引都增加一个维度
print(index_train.shape, index_val.shape)     

dataset_train = tf.data.Dataset.from_tensor_slices((image_train, index_train))#加载训练dataset，一个是train_path,另外一个是train_index
dataset_val = tf.data.Dataset.from_tensor_slices((image_val, index_val))#加载测试dataset

(1899, 1) (475, 1)


In [10]:
def load_image(path,label):   #函数用来获取图片并作数据增强
    image = tf.io.read_file(path)   #这个是读取并输出文件名的全部内容
    image = tf.image.decode_jpeg(image, channels=3)  #对图片进行解码，即把图片转化成tensor,图片是RGB三通道的所以channels=3
    image = tf.image.resize(image, [64, 64])   #将图片裁剪成64X64的图片，默认的裁剪方式一般是双线性插值，也可以选择其他方式但要在工程里修改
    # 数据增强(随机翻转)
    image = tf.image.random_flip_left_right(image)  #将图片进行随机左右旋转，拓展训练集数量，用来减轻过拟合现象
    image = tf.image.random_flip_up_down(image)    #对图片进行随机上下旋转
    # image = tf.image.random_crop(image, [64, 64, 3])
    image = tf.cast(image, tf.float32)/255   #数据归一化，1.提高训练速度，2.提高模型精度，具体请百度，之所以除以255，是因为图片像素是从0到255
    label = tf.convert_to_tensor(label)    #将标签转化成tensor
    return image, label

In [11]:
BATCH_SIZE = 16   #这里batch_size是16，每次读取图片数量是16张，和显存有关,
AUTOTUNE = tf.data.experimental.AUTOTUNE #多线程设置，自动模式，使用多线程进行训练
dataset_train = dataset_train.map(load_image, num_parallel_calls=AUTOTUNE)
dataset_train = dataset_train.batch(BATCH_SIZE)
dataset_val = dataset_val.map(load_image, num_parallel_calls=AUTOTUNE)
dataset_val = dataset_val.batch(BATCH_SIZE)

训练模型

In [14]:
# create CNN
#使用卷积神经网络，构建模型都是使用tensorflow的layers类来进行构建的
def CNNmodel(input_shape,filters=64, kernel=(3,3),size=4,dropout=0.2,**kwargs):
    #input_shape是输入图片尺寸，因为之前做了resize，所以这里都是64乘64乘3的图片，
    #filters是卷积核数量，kernel是卷积核大小，dropout是随机丢弃方法，0.2是随机丢弃的概率，
    #为了可以在for循环中多执行几次
    _inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(8,(3,3),padding='same',use_bias=False,strides=(2,2), name='conv_0')(_inputs)
    #二维卷积，不太明白的是第一个参数为什么是8？这个参数在API中对应filters,也就是输出通道数，图片原先通道数是3现在变成了8
    #padding='same',所以这里输出图片尺寸32*32，
    x = layers.BatchNormalization(axis=-1, name='conv_0_bn')(x)#加了一个batchnorm
    x = layers.ReLU(6., name='conv_0_relu')(x)#加上了Relu激活函数，增加非线性
    
    x = layers.AveragePooling2D(name='avg_1')(x)#平均池化层，浅显来说就是将图片长、宽减半的层，具体的作用方式和卷积造成的图片长宽减半不同，目的是降维
    #平均池化后，图片尺寸变成了16乘以16，通道数和上面一致，为8
    x = layers.Dropout(dropout, name='dropout_1')(x)#随机丢弃层

    for block_id in range(2,size+2):#多卷积几次，卷积的目的实际上是提取特征，一般来说，卷积之后是要做batchnorm，添加激活函数，然后做dropout
        x = layers.Conv2D(filters,kernel,padding='same',use_bias=False,strides=(1,1), name='conv_%d'%block_id)(x)
        #这里的卷积应该是用来做特征提取的，for循环最后输出尺寸是16乘以16，通道数是8
        x = layers.BatchNormalization(axis=-1, name='conv_%d_bn'%block_id)(x)#batchnorm
        x = layers.ReLU(6., name='conv_%d_relu'%block_id)(x)#添加激活函数

    x = layers.Conv2D(64,(3, 3),padding='same',use_bias=False,strides=(2,2), name='conv_1')(x)
    #卷积核数量64，意味输出通道数是64，padding='same'，所以现在输出图片尺寸是8乘8，通道数是64
    x = layers.AveragePooling2D()(x)#又做了一层平均池化，所以图片尺寸是4乘4，通道数是64
    x = layers.Dropout(dropout, name='dropout_2')(x)
    x = layers.GlobalAveragePooling2D(name='avg_2')(x)#全局平均池化，卷积提取特征最后一步
    x = layers.Dropout(dropout, name='dropout_0')(x)
    x = layers.Dense(4)(x)#全连接层，分类作用
    x = layers.Softmax()(x)#softmax函数，将softmax用于多分类过程中，将多个输入映射到（0,1）区间内
    return keras.Model(inputs=_inputs,outputs=x)

In [15]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='accuracy', factor=0.5, patience=4, min_lr=0.0001,verbose=1)
earlystop = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=8,verbose=1)#这两行主要作用是检测训练的过程

model = CNNmodel(input_shape=(64,64,3),filters=64, kernel=(3,3),size=3)#定义模型参数
model.compile(optimizer='SGD',loss='sparse_categorical_crossentropy',metrics=['accuracy'])#使用SGD优化器，损失函数，评价指标accuracy

In [16]:
history = model.fit(dataset_train,validation_data=dataset_val,callbacks=[reduce_lr],verbose=2,epochs=1)#模型训练

119/119 - 4s - loss: 1.0877 - accuracy: 0.5271 - val_loss: 1.4423 - val_accuracy: 0.2358 - lr: 0.0100


In [ ]:
plt.plot(history.history['val_accuracy'],label='val_acc')#这个是用来测试集的准确度提升曲线图，x轴是轮数，y轴是accuracy（准确度）
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Acc')
plt.show()

In [ ]:
model.save('model.h5')#保存模型

模型读取+验证

In [ ]:
import cv2

In [ ]:
keras_file = 'model.h5'
model_read = tf.keras.models.load_model(keras_file)#把模型导入

In [ ]:
test_path = glob.glob('Dataset-test/Recyclable_trash/*')#测试的文件路径
num=0
for n in range(30):#选前30张图片来进行测试
    print(test_path[n])
    image = cv2.imread(test_path[n])#读取图像，里面参数是图片的路径
    image = cv2.resize(image, (64, 64))#将读取的图片resize成64乘64的大小
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)#因为opencv读取出来的是BGR图像，转化成RGB图像
    image_bn = image.astype("float32") / 255.0#数据归一化
    image = np.expand_dims(image, axis=0)#拓展图片维度
    image_bn = np.expand_dims(image_bn, axis=0)
    pred = model.predict(image_bn)#使用模型进行预测
    max_index = np.argmax(pred)#将模型对4类的预测中，概率最大的数值提取出来
    print(max_index)
    if max_index==1:
        print('厨余垃圾')
        num+=1
    elif max_index==3:
        print('可回收垃圾')       
    elif max_index==2:
        print('其他垃圾') 
    else:
        print('有害垃圾')
        num+=1
print(num/30)

数据集重命名

In [ ]:
import os
path='Dataset/Recyclable_trash'    

#获取该目录下所有文件，存入列表中
fileList=os.listdir(path)
n=0
for i in fileList:
    
    #设置旧文件名（就是路径+文件名）
    oldname=path+ os.sep + fileList[n]   # os.sep添加系统分隔符
    
    #设置新文件名
    newname=path + os.sep +'rec_gla'+str(n+1)+'.jpeg'
    
    os.rename(oldname,newname)   #用os模块rename方法对文件改名
    n+=1

文件异常检测

In [ ]:
# 文件可能报错，提示一堆格式错误
# 修改数据集路径dir，和错误中出现格式如：BM
import os
dir = 'Dataset/Recyclable_trash/'
for i, filename in enumerate(os.listdir(dir)):
    filename = dir + filename
    with open(filename, 'rb') as imageFile:
        if imageFile.read().startswith(b'BM'):
            print(f"{i}: {filename} - found!")